In [1]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
#import soundfile as sf
from IPython.display import Audio

Flash attention 2 is not installed


In [7]:
import os
# Set device currently does nothing
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"
device='cpu'

In [8]:
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [106]:
prompt = "Hey, how are you doing today?"
description = """Gary's with a french accent delivers a slightly expressive and animated speech with a 
moderate speed and pitch. The recording is of very high quality, with the speaker's voice 
sounding clear and very close up."""


In [107]:
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

In [108]:
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
# mps generates errors: this is a work around that required modifying the transformers util source code
# lib/python3.XX/site-packages/transformers/generation/utils.py
# https://github.com/huggingface/transformers/issues/31744

audio_arr = generation.cpu().numpy().squeeze()

In [109]:
Audio(audio_arr, rate=model.config.sampling_rate)